- you're going to learn about behavioral cloning, but more importantly, you're going to play with simulation
- you're going to drive a car through simulating yourself, and then you make a neural network copy you
- the neural network will watch you--your behavior
- the task for the neural network is to pick up your driving skill, and copy it and do the same
- there's two ways to fail
  - first is you know a network fails
  - second is that you are a lousy driver--then find somebody else to drive for you

-  we encourage you to drive the vehicle in training mode and collect your own training data, but we have also included [sample driving data](https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip) for the first track, which you can optionally use to train your network
- you may need to collect additional data in order to get the vehicle to stay on the road

# Running the Simulator

- here are the latest updates to the simulator:
  - steering is controlled via position mouse instead of keyboard
    - this creates better angles for training--note the angle is based on the mouse distance
    - to steer, hold the left mouse button and move left or right
    - to reset the angle to 0 simply lift your finger off the left mouse button
  - you can toggle record by pressing R, previously you had to click the record button (you can still do that)
  - when recording is finished, saves all the captured images to disk at the same time instead of trying to save them while the car is still driving periodically
    - you can see a save status and play back of the captured data
  - you can takeover in autonomous mode
    - while W or S are held down you can control the car the same way you would in training mode
    - this can be helpful for debugging
    - as soon as W or S are let go, autonomous takes over again
  - pressing the spacebar in training mode toggles on and off cruise control (effectively presses W for you)
  - added a Control screen
  - track 2 was replaced from a mountain theme to Jungle with free assets; note the track is challenging
  - you can use brake input in `drive.py` by issuing negative throttle values


- if you are interested here is the source code for the [simulator repository](https://github.com/udacity/self-driving-car-sim)


- when you first run the simulator, you’ll see a configuration screen asking what size and graphical quality you would like
- we suggest running at the smallest size and the fastest graphical quality
- we also suggest closing most other applications (especially graphically intensive applications) on your computer, so that your machine can devote its resources to running the simulator

## Training Mode

- the next screen gives you two options: Training Mode and Autonomous Mode
- select Training Mode
  - the simulator will load and you will be able to drive the car like it’s a video game
- you'll use autonomous mode in a later step, after you've used the data you collect here to train your neural network

# Collecting Training Data

- for the behavioral cloning project, you only need to work with this first track
- but if you want to give yourself an extra challenge, you can also train your vehicle to drive on the second track
- steering with the mouse actually outputs the best steering measurements and the training data


- in order to start collecting training data, you'll need to do the following:
  - enter Training Mode in the simulator
  - start driving the car to get a feel for the controls
  - when you are ready, hit the record button in the top right to start recording
  - continue driving for a few laps or till you feel like you have enough data
  - hit the record button in the top right again to stop recording

- some of the interesting features of the track will also present challenges for training an end-to-end driving model
  - for example, the model will have to learn how to handle these sharp turns
  - this bridge has a different texture and different borders than the other parts of the road
  - the dirt border on the edge of the track here is different than the curbs that lined the rest of the track--that could be tricky for the model to learn
  - this sequence of sharp turns is challenging to drive even by hand--the neural network could easily lose control here and drive the car into the water
- once we've gotten past this second dirt border, we've completed about one lap of driving

- so that the car drives down the center of the road, it's essential to capture center lane driving
- try driving around the track various times while staying as close to the middle of the track as possible even when making turns
- in the real world, the car would need to stay in a lane rather than driving down the center but for the purposes of this project, aim for center of the road driving


- now that you have driven the simulator and know how to record data, it's time to think about collecting data that will ensure a successful model
- there are a few general concepts to think about that we will later discuss in more detail:
  - the car should stay in the center of the road as much as possible
  - if the car veers off to the side, it should recover back to center
  - driving counter-clockwise can help the model generalize
  - flipping the images is a quick way to augment the data
  - collecting data from the second track can also help generalize the model
  - we want to avoid overfitting or underfitting when training the model
  - knowing when to stop collecting more data

# Data Visualization

- the simulator data consists primarily of images
- before I start using this data to train a model, I'd like to look at those images
- I'll start by opening `driving_log.csv`
  - each line in this file represents a point in time during my training lap, and each line has seven tokens
    - the first three tokens in each line are paths to images, one each for cameras mounted on the center, left, and right of the vehicle's windshield
    - the next four tokens are measurements for steering, throttle, brake, and speed
      - the steering measurements range between $-1$ and $1$
      - the throttle measurements range between $0$ and $1$
      - the brake measurements seem to all be $0$,
      - the speed measurements range from $0$ to $30$
  - the throttle, brake, and speed data could all be useful for training the network, but for now, I'm going to ignore them


- I'll start by using the images as the feature set and the steering measurements as the label set
- then I'll use the images to train the network to predict the steering measurements


- you might wonder why the simulator gives us three camera shots for each point in time
  - the reason is that we can use these camera shots to help us generalize the model
- effectively, we can pass these images to the network to teach the network what it looks like to be off the center of the road and how to steer to get back to the center

# Training Your Network

- the image data from the simulator is going to be much easier to work with on a GPU than on a CPU
- I'll use Python CSV library to read and store the lines from the `drivinglog.csv` file
- once I have the current path, I can use OpenCV to load the image and once I've loaded the image, I can append it to my list of images


- I can do something similar for the steering measurements, which will serve as my output labels
- it's actually going to be easier to load this steering measurements, because there are no paths or images to handle
- I simply extract the fourth token from the CSV line, and then cast it as a float
- that gives me the steering measurement for this point in time
- then, I append that measurement to the larger measurements array, just like I did for the image


- now that I've loaded the images and steering measurements, I'm going to convert them to NumPy arrays, since that's the format Keras requires


- for the loss function, I'll use mean squared error, or MSE
- this is different than the cross-entropy function we've used in the past, again because this is a regression network instead of a classification network
- what I want to do is minimize the error between the steering measurement that the network predicts and the ground truth steering measurement
- mean squared error is a good loss function for this


- once the model is compiled, I'll train it with the feature and label arrays I just built
- I'll also shuffle the data and split off 20% of the data to use for a validation set
- by default, Keras trains for 10 epoches


- finally, I'm going to save the trained model, so that later I can download it onto my local machine, and see if it works for driving the simulator
- next, I'll download this model to my local machine, and see how well it drives the car in the simulator

- note: `cv2.imread` will get images in BGR format, while `drive.py` uses RGB
- in the video above one way you could keep the same image formatting is to do `image = ndimage.imread(current_path)` with `from scipy import ndimage` instead

## Training Your Network

- now that you have training data, it’s time to build and train your network!
- use Keras to train a network to do the following:
  - take in an image from the center camera of the car--this is the input to your neural network
  - output a new steering angle for the car
- you don’t have to worry about the throttle for this project, that will be set for you


- [save your trained model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model) architecture as `model.h5` using `model.save('model.h5')`

# Running Your Network

- in order to run the simulator in autonomous mode on my local machine, I'm going to first clone the GitHub repo for the project
- this repo provides the `drive.py` file that will load my saved model and use it to make steering predictions
- I'll run `drive.py` and pass my model file as an argument
  - `drive.py` is running now, waiting for the simulator to start in autonomous mode


- when I launch the simulator this time, I'll select autonomous mode
- I can see from the output in the terminal that `drive.py` is successfully predicting steering measurements based on my model

## Validating Your Network

- in order to validate your network, you'll want to compare model performance on the training set and a validation set
- the validation set should contain image and steering data that was not used for training
- a rule of thumb could be to use $80\%$ of your data for training and $20\%$ for validation or $70\%$ and $30\%$
- be sure to randomly shuffle the data before splitting into training and validation sets


- if model predictions are poor on both the training and validation set (for example, mean squared error is high on both), then this is evidence of underfitting
- possible solutions could be to
  - increase the number of epochs
  - add more convolutions to the network


- when the model predicts well on the training set but poorly on the validation set (for example, low mean squared error for training set, high mean squared error for validation set), this is evidence of overfitting
- if the model is overfitting, a few ideas could be to
  - use dropout or pooling layers
  - use fewer convolution or fewer fully connected layers
  - collect more data or further augment the data set


- ideally, the model will make good predictions on both the training and validation sets
- the implication is that when the network sees an image, it can successfully predict what angle was being driven at that moment

## Testing Your Network

- once you're satisfied that the model is making good predictions on the training and validation sets, you can test your model by launching the simulator and entering autonomous mode
- the car will just sit there until your Python server connects to it and provides it steering angles


- here’s how you start your Python server: `python drive.py model.h5`


- once the model is up and running in `drive.py`, you should see the car move around (and hopefully not off) the track!
- if your model has low mean squared error on the training and validation sets but is driving off the track, this could be because of the data collection process
- it's important to feed the network examples of good driving behavior so that the vehicle stays in the center and recovers when getting too close to the sides of the road

# Data Preprocessing

- the first obvious step to improve the model is to preprocess the data
- I'm going to add two preprocessing steps, *normalizing the data* and *mean centering the data*


- for normalization, I'll add a Lambda layer to my model
- within this Lambda layer, I'll normalize the image by dividing each element by $255$, which is the maximum value of an image pixel
- once the image is normalized to a range between $0$ and $1$, I'll mean center the image by subtracting $0.5$ from each element, which will shift the element mean down from $0.5$ to $0$


- when I train this model, now I can see that the training loss and the validation loss are both much smaller, which is a good sign
- however, if the validation loss only decreases for the first few epoches and then starts oscillating up and down, that's a sign that the model may be overfitting the training data
  - I'll reduce the number of epoches and train the model again

## Lambda Layers

- in Keras, [lambda layers](https://keras.io/layers/core/#lambda) can be used to create arbitrary functions that operate on each image as it passes through the layer


- in this project, a lambda layer is a convenient way to parallelize image normalization
- the lambda layer will also ensure that the model will normalize input images when making predictions in `drive.py`
- that lambda layer could take each pixel in an image and run it through the formulas:
  - `pixel_normalized = pixel / 255`
  - `pixel_mean_centered = pixel_normalized - 0.5`
- a lambda layer will look something like: 
  - `Lambda(lambda x: (x / 255.0) - 0.5)`


- below is some example code for how a lambda layer can be used

```python
from keras.models import Sequential, Model
from keras.layers import Lambda

# set up lambda layer
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
...
```

# Data Augmentation

- I have this problem that the car seems to pull too hard to the left
- this actually makes sense--the training track is a loop, and the car drives counter-clockwise
- so most of the time, the model is learning to steer to the left--then in autonomous mode, the model does steer to the left even in situations when staying straight might be best
- one approach to mitigate this problem is data augmentation
- there are many ways to augment data to expand the training set and help the model generalize better
  - I could change the brightness on the images or I could shift them horizontally or vertically
  - in this case, though, I'm going to keep things pretty simple--I'm just going to flip the images horizontally like a mirror
  - then I'll invert the steering angles, and I should wind up with a balanced data set that teaches the car to steer clockwise, as well as counter-clockwise


- just like with using side camera data, using data augmentation carries two benefits
  - we have more data to use for training the network
  - the data we use for training the network is more comprehensive

## Flipping Images And Steering Measurements

- an effective technique for helping with the left turn bias involves flipping images and taking the opposite sign of the steering measurement

```python
import numpy as np
image_flipped = np.fliplr(image)
measurement_flipped = -measurement
```

- the cv2 library also has similar functionality with the [flip method](http://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html#flip)

# Using Multiple Cameras

- using the side camera images carries two benefits
  - it's simply more data--in fact, it's three times as much data
  - using these images will help teach the network how to steer back to the center if the vehicle starts drifting off to the side


- let's look at the left image--this image is a little off to the left side of the road, but it's paired with a steering measurement of zero
- that's because when this image was taken, the center of the vehicle really was in the center of the road


- basically, I want to train the network to steer a little harder to the right whenever the network sees an image like this
  - I'll do this by taking the actual steering measurement, which in this case is zero, and adding a small correction factor to it
  - I could actually draw out a model and use trigonometry and physics to calculate an optimal correction factor, but I'm just going to be lazy and try a correction factor of $0.2$


- similarly, for the right camera image, I'm going to train the network to steer a little harder to the left
- I'll do that by subtracting the correction factor from the actual steering measurement whenever I use a right camera image
  - I implement this by looping through the first three tokens of each line in the CSV file, and I use these to load each camera image into the images array
  - then I add three measurements to the measurements array corresponding to each image I just added


- now I should have three times as much data, and my model should perform better

- the simulator captures images from three cameras mounted on the car: a center, right and left camera
- that’s because of the issue of recovering from being off-center
- in the simulator, you can weave all over the road and turn recording on and off to record recovery driving
- in a real car, however, that’s not really possible--at least not legally


- so in a real car, we’ll have multiple cameras on the vehicle, and we’ll map recovery paths from each camera
  - for example, if you train the model to associate a given image from the center camera with a left turn, then you could also train the model to associate the corresponding image from the left camera with a somewhat softer left turn
  - and you could train the model to associate the corresponding image from the right camera with an even harder left turn
- in that way, you can simulate your vehicle being in different positions, somewhat further off the center line


- to read more about this approach, see [this paper](http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf) by our friends at NVIDIA that makes use of this technique

## Explanation of How Multiple Cameras Work

- the image below gives a sense for how multiple cameras are used to train a self-driving car
- this image shows a bird's-eye perspective of the car
- the driver is moving forward but wants to turn towards a destination on the left


- from the perspective of the left camera, the steering angle would be less than the steering angle from the center camera
- from the right camera's perspective, the steering angle would be larger than the angle from the center camera
- the next section will discuss how this can be implemented in your project although there is no requirement to use the left and right camera images

<img src="resources/using_multiple_cameras.png"/>

## Multiple Cameras in This Project

- for this project, recording recoveries from the sides of the road back to center is effective
- but it is also possible to use all three camera images to train the model
- when recording, the simulator will simultaneously save an image for the left, center and right cameras
- each row of the csv log file, `driving_log.csv`, contains the file path for each camera as well as information about the steering measurement, throttle, brake and speed of the vehicle


- here is some example code to give an idea of how all three images can be used:

```python
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            steering_center = float(row[3])

            # create adjusted steering measurements for the side camera images
            correction = 0.2 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction

            # read in images from center, left and right cameras
            path = "..." # fill in the path to your training IMG directory
            img_center = process_image(np.asarray(Image.open(path + row[0])))
            img_left = process_image(np.asarray(Image.open(path + row[1])))
            img_right = process_image(np.asarray(Image.open(path + row[2])))

            # add images and angles to data set
            car_images.extend(img_center, img_left, img_right)
            steering_angles.extend(steering_center, steering_left, steering_right)
```

- during training, you want to feed the left and right camera images to your model as if they were coming from the center camera
  - this way, you can teach your model how to steer if the car drifts off to the left or the right
- figuring out how much to add or subtract from the center angle will involve some experimentation
- during prediction (i.e. "autonomous mode"), you only need to predict with the center camera image
- it is not necessary to use the left and right images to derive a successful model
- recording recovery driving from the sides of the road is also effective

# Cropping Images in Keras

- I'd like to return to something I saw a while back, which is that the top pixels of the image mostly capture sky and trees and hills and other elements that might be more distracting for the model than helpful
  - it looks like this is about 70 rows of pixels
- I also notice that the bottom 25 pixels largely consist of the hood of the car
- I'm going to crop out those portions of the images.


- just like with the Lambda layer, I'm going to use a built-in Keras layer to perform the cropping inside of the model
- in this case I'll use the Cropping 2D layer to remove the top 70 pixels and the bottom 25 pixels

- your model might train faster if you crop each image to focus on only the portion of the image that is useful for predicting a steering angle

## Cropping2D Layer

- Keras provides the [Cropping2D layer](https://keras.io/layers/convolutional/#cropping2d) for image cropping within the model
- this is relatively fast, because the model is parallelized on the GPU, so many images are cropped simultaneously
- by contrast, image cropping outside the model on the CPU is relatively slow
- also, by adding the cropping layer, the model will automatically crop the input images when making predictions in `drive.py`
- the Cropping2D layer might be useful for choosing an area of interest that excludes the sky and/or the hood of the car


- cropping Layer Code example:

```python
from keras.models import Sequential, Model
from keras.layers import Cropping2D
import cv2

# set up cropping2D layer
model = Sequential()
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
...
```
- the example above crops:
  - 50 rows pixels from the top of the image
  - 20 rows pixels from the bottom of the image
  - 0 columns of pixels from the left of the image
  - 0 columns of pixels from the right of the image

# More Data Collection

- one obvious approach to improve the network that I haven't tried yet--maybe THE obvious approach, is to gather more data
- I could also try to be more clever about data collection
- for example, I could drive around the track in the opposite direction
  - training in the opposite direction is kind of like gathering data from a whole new track, and this data will help my model generalize better
  - so when I run in autonomous mode, if the vehicle gets into a weird place on the road, the model will have a better chance of making a good decision
- in the same vein, I could gather training data from multiple tracks and mix them together
  - this could produce a much more generalized model that can drive either track successfully

### Recovery Laps

- if you drive and record normal laps around the track, even if you record a lot of them, it might not be enough to train your model to drive properly
- here’s the problem: if your training data is all focused on driving down the middle of the road, your model won’t ever learn what to do if it gets off to the side of the road
- and probably when you run your model to predict steering measurements, things won’t go perfectly and the car will wander off to the side of the road at some point


- so you need to teach the car what to do when it’s off on the side of the road
  - one approach might be to constantly wander off to the side of the road and then steer back to the middle
  - a better approach is to only record data when the car is driving from the side of the road back toward the center line
  - so as the human driver, you’re still weaving back and forth between the middle of the road and the shoulder, but you need to turn off data recording when you weave out to the side, and turn it back on when you steer back to the middle

### Driving Counter-Clockwise

- track one has a left turn bias
- if you only drive around the first track in a clock-wise direction, the data will be biased towards left turns
- one way to combat the bias is to turn the car around and record counter-clockwise laps around the track
- driving counter-clockwise is also like giving the model a new track to learn from, so the model will generalize better

### Using Both Tracks

- if you end up using data from only track one, the convolutional neural network could essentially memorize the track
- consider using data from both track one and track two to make a more generalized model

### Collecting Enough Data

- how do you know when you have collected enough data?
- machine learning involves trying out ideas and testing them to see if they work
- if the model is over or underfitting, then try to figure out why and adjust accordingly
- since this model outputs a single continuous numeric value, one appropriate error metric would be mean squared error
  - if the mean squared error is high on both a training and validation set, the model is underfitting
  - if the mean squared error is low on a training set but high on a validation set, the model is overfitting
    - collecting more data can help improve a model when the model is overfitting


- what if the model has a low mean squared error on both the training and validation sets, but the car is falling off the track?
  - try to figure out the cases where the vehicle is falling off the track
  - does it occur only on turns?
    - then maybe it's important to collect more turning data
  - the vehicle's driving behavior is only as good as the behavior of the driver who provided the data


- here are some general guidelines for data collection:
  - two or three laps of center lane driving
  - one lap of recovery driving from the sides
  - one lap focusing on driving smoothly around curves

# Visualizing Loss

### Outputting Training and Validation Loss Metrics

- in Keras, the `model.fit()` and `model.fit_generator()` methods have a verbose parameter that tells Keras to output loss metrics as the model trains
- the `verbose` parameter can optionally be set to `verbose = 1` or `verbose = 2`
- setting `model.fit(verbose = 1)` will:
  - output a progress bar in the terminal as the model trains.
  - output the loss metric on the training set as the model trains.
  - output the loss on the training and validation sets after each epoch
- with `model.fit(verbose = 2)`, Keras will only output the loss on the training set and validation set after each epoch

### Model History Object

- when calling `model.fit()` or `model.fit_generator()`, Keras outputs a history object that contains the training and validation loss for each epoch
- the following code shows how to use the `model.fit()` history object to produce the visualization

```python
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()
```

# Generators

- the images captured in the car simulator are much larger than the images encountered in the Traffic Sign Classifier Project, a size of $160 x 320 x 3$ compared to $32 x 32 x 3$
- storing $10 000$ traffic sign images would take about $30$ MB but storing $10 000$ simulator images would take over $1.5$ GB
- that's a lot of memory!--not to mention that preprocessing data can change data types from an int to a float, which can increase the size of the data by a factor of $4$

- generators can be a great way to work with large amounts of data
- instead of storing the preprocessed data in memory all at once, using a generator you can pull pieces of the data and process them on the fly only when you need them, which is much more memory-efficient
- a generator is like a [coroutine](https://en.wikipedia.org/wiki/Coroutine), a process that can run separately from another main routine, which makes it a useful Python function


- instead of using return, the generator uses yield, which still returns the desired output values but saves the current values of all the generator's variables
- when the generator is called a second time it re-starts right after the yield statement, with all its variables set to the same values as before

- below is a short quiz using a generator
- this generator appends a new Fibonacci number to its list every time it is called
- the result will be we can get the first $10$ Fibonacci numbers simply by calling our generator $10$ times
- if we need to go do something else besides generate Fibonacci numbers for a while we can do that and then always just call the generator again whenever we need more Fibonacci numbers

In [1]:
def fibonacci():
    numbers_list = []
    while 1:
        if(len(numbers_list) < 2):
            numbers_list.append(1)
        else:
            numbers_list.append(numbers_list[-1] + numbers_list[-2])
        yield numbers_list

our_generator = fibonacci()
my_output = []

for i in range(10):
    my_output = (next(our_generator))
    
print(my_output)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


- here is an example of how you could use a generator to load data and preprocess it on the fly, in batch size portions to feed into your Behavioral Cloning model

```python
import os
import csv

samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

ch, row, col = 3, 80, 320  # Trimmed image format

model = Sequential()
# Preprocess incoming data, centered around zero with small standard deviation 
model.add(Lambda(lambda x: x/127.5 - 1.,
        input_shape=(ch, row, col),
        output_shape=(ch, row, col)))
model.add(... finish defining the rest of your model architecture here ...)

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, /
            steps_per_epoch=ceil(len(train_samples)/batch_size), /
            validation_data=validation_generator, /
            validation_steps=ceil(len(validation_samples)/batch_size), /
            epochs=5, verbose=1)
```

# Recording Video in Autonomous Mode

- because your hardware setup might be different from a reviewer's hardware setup, driving behavior could be different on your machine than on the reviewer's
- to help with reviewing your submission, we require that you submit a video recording of your vehicle driving autonomously around the track
- the video should include at least one full lap around the track. Keep in mind the rubric specifications:
  - "No tire may leave the drivable portion of the track surface. The car may not pop up onto ledges or roll over any surfaces that would otherwise be considered unsafe (if humans were in the vehicle)."


- in the [GitHub repo](https://github.com/udacity/CarND-Behavioral-Cloning-P3), we have included a file called `video.py`, which can be used to create the video recording when in autonomous mode
- the README file in the GitHub repo contains instructions about how to make the video recording
  - here are the instructions as well: `python drive.py model.h5 run1`
  - the fourth argument, `run1`, is the directory in which to save the images seen by the agent
    - if the directory already exists, it'll be overwritten


- the image file name is a timestamp of when the image was seen
- this information is used by `video.py` to create a chronological video of the agent driving

### Using video.py

- `python video.py run1` creates a video based on images found in the run1 directory
- the name of the video will be the name of the directory followed by `'.mp4'`, so, in this case the video will be `run1.mp4`
- optionally, one can specify the FPS (frames per second) of the video: `python video.py run1 --fps 48`
  - the video will run at 48 FPS
  - the default FPS is 60

# Further Help

- a former student has written [a helpful guide](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/Behavioral+Cloning+Cheatsheet+-+CarND.pdf) for those of you looking for some hints and advice